In [12]:
!pip install simpy

### Componentes de un modelo

Podemos dividir los componentes de un modelo en:

• __Componentes activos__

Son los protagonistas de nuestro modelo, son los que
interactuando con los distintos recursos de nuestro sistema, le
dan vida al mismo.
Por ejemplo:

Clientes, autos, instrucciones de un procesador

• __Componentes pasivos o recursos__

Son los componentes que nuestros componentes activos
utilizarán, “no tienen vida propia”.

Por ejemplo:

Un empleado, semáforos, un procesador

### Componentes activos

En Simpy, el comportamiento de los componentes activos es
modelado utilizando funciones generadoras.
Interactúan con el ambiente y con otros componentes, mediante
eventos.


### Funciones generadoras

Son funciones que devuelven un iterador que puede ser utilizado en
un ciclo.
Utilizan la expresión yield con la cual se suspende la ejecución de la
función en ese punto, guardando el ambiente de la misma:

• Línea de ejecución

• Variables locales

• Try-statements

In [13]:
import simpy as simpy

In [14]:
def pares():
    index=1
    while index<=10:
        yield index *2
        index=index+1

In [15]:
for i in pares():
    print (i)

2
4
6
8
10
12
14
16
18
20


### Ambiente

Es el encargado de manejar el tiempo de simulación, como así
también la programación y ejecución de los eventos.

Clases: BaseEnvironment, Environment, RealTimeEnvironment

Guarda los eventos en una lista (_queue) ordenados por:

• Prioridad

• Momento en el cual deben volver a actuar

• Event Id decreciente

Método más importante

__Environment.run(until=None)__

Environment nos brinda algunas propiedades para conocer su
estado:

• __Environment.now__ : Devuelve el momento actual de simulación.

• __Environment.peek()__: Devuelve el próximo momento en el que
ocurrirá un evento.

• __Environment.active_process:__ Devuelve “None” o apunta al
proceso actualmente activo. Un proceso está activo cuando su
función de proceso se está ejecutando, se vuelve inactivo (o
suspendido) cuando se produce un evento.

### Eventos

Existen diversas formas de sincronizar procesos, una de ellas es el
lanzamiento de eventos

Cuando un evento es lanzado, se programa para ocurrir en un
determinado instante de tiempo y es insertado en la lista de eventos
del ambiente de simulación (event queue).

El proceso que lanzó dicho proceso se suspende y será reanudado cuando el evento generado ocurra.
En caso de que múltiples procesos esperen el mismo evento, serán reanudados en el mismo orden en que produjeron ese evento.

Pueden estar en alguno de los siguientes estados:

• No lanzado

• Lanzado

• Procesado

### Control de la simulación

Toda simulación debe tener un criterio por el cual finalizar

• Simular hasta llegar a un momento determinado
Simular 10 horas

• Simular hasta que no queden eventos por ejecutar
Simular hasta que todos los clientes se hayan retirado de una
oficina

• Simular hasta que se ejecute un evento en particular
Simular hasta que la cantidad de requerimientos recibidos por un
servidor lleguen a 1000

In [16]:
def caminante(env):
    while True:
        t_caminata = 10
        yield env.timeout(t_caminata)
        print('Fin caminata a los %d min' % env.now)
        t_descanso = 2
        yield env.timeout(t_descanso)
        print('Fin descanso a los %d min' % env.now)

In [24]:
env = simpy.Environment()
env.process(caminante(env))
env.run(until=24) #simula 24 unidades de tiempo

Fin caminata a los 10 min
Fin descanso a los 12 min
Fin caminata a los 22 min


In [18]:
def reloj(env):
    yield env.timeout(24)

In [21]:
env = simpy.Environment()

env.process(caminante(env))
env.run(env.process(reloj(env))) #simula por tiempo utilizando eventos

Fin caminata a los 10 min
Fin descanso a los 12 min
Fin caminata a los 22 min
Fin descanso a los 24 min


In [32]:
from random import randint

In [35]:
class semaforo:
    def __init__(self, env):
        self.env = env
        
    def verde(self):
        yield self.env.timeout(randint(40,70))
        print('Semaforo verde a los %d min' % env.now)

In [36]:
class peaton:
    def __init__(self, env):
        self.env = env
        
    def cruzar(self):
        yield self.env.timeout(randint(45,55))
        print('Cruce de peaton a los %d min' % env.now)

In [37]:
class auto:
    def __init__(self, env, s, p):
        self.env = env
        self.drive_proc = env.process(self.manejar(s,p))
        
    def manejar(self, s, p):
        cruce=self.env.process(p.cruzar())
        semaf=self.env.process(s.verde())
        yield cruce & semaf #espera a que terminen los dos eventos
        print('Avanza auto a los %d min' % env.now)

In [38]:
env = simpy.Environment()

p=peaton(env)
s=semaforo(env)

mi_auto = auto(env,s,p)
env.run()

Cruce de peaton a los 48 min
Semaforo verde a los 60 min
Avanza auto a los 60 min


### Resources

Pueden ser usados por un número limitado de procesos a la vez

Creacion: Resource(environment, capacity=1)

Los componentes activos o procesos:

• __Solicitan (request)__ el recurso y se convierten en dueños,
haciendo uso del mismo.

• __Liberan (release)__ el recurso, una vez que los hayan terminado
de utilizar.

Ejemplo __liberacion manual__

In [42]:
empleado = simpy.Resource(env,capacity=1)

In [43]:
def atencion(env, empleado):
    t_atencion = 60
    emp = empleado.request()
    yield emp #espero el acceso al recurso
    yield env.timeout(t_atencion)
    emp.release()
    print('Fin atención a los %d min' % env.now)

In [ ]:
env = simpy.Environment()
empleado = simpy.Resource(env,capacity=1)
env.process(atencion(env,empleado))
env.run()

Ejemplo __liberacion automatica__

In [45]:
def atencion(env, empleado):
    t_atencion = 60
    with empleado.request() as emp:
        yield emp
        yield env.timeout(t_atencion)
        print('Fin atención a los %d min' % env.now)

In [46]:
env = simpy.Environment()
empleado = simpy.Resource(env,capacity=1)
env.process(atencion(env,empleado))
env.run()

Fin atención a los 60 min


Algunas propiedades de los recursos

Resources nos brindan algunas propiedades para conocer su estado:

• __Resource.count:__ retorna la cantidad actual de usuarios usando el
recurso.

• __Resource.capacity:__ retorna la capacidad del recurso.

• __Resource.users:__ retorna los usuarios actuales que estén usando
el recurso.

• __Resource.queue:__ retorna los usuarios que se encuentran
esperando en la cola para hacer uso del recurso.